In [2]:
import FormulaTools as ft
import Progression as prog
from Constants import *
import numpy as np

PathToDataFile = ''
SampleSignal = ImportSampleData(PathToDataFile)


Formulas, Prob = ImportCandidateFormulas()
idx = np.argsort(Prob)[::-1]
Formulas = np.array(Formulas)
Prob = np.array(Prob)
Formulas = Formulas[idx]
Prob = Prob[idx]
formula = Formulas[0]
print(formula)

['and', ['G', ['and', ['not', ['T0']], ['not', ['T1']], ['not', ['T2']], ['not', ['T3']], ['not', ['T4']]]], ['imp', ['P0'], ['F', ['W0']]], ['imp', ['P1'], ['F', ['W1']]], ['imp', ['P2'], ['F', ['W2']]], ['imp', ['P3'], ['F', ['W3']]], ['imp', ['P0'], ['U', ['not', ['W1']], ['W0']]], ['imp', ['P0'], ['U', ['not', ['W2']], ['W0']]], ['imp', ['P1'], ['U', ['not', ['W2']], ['W1']]]]


Now first lets verify that the syntax of the sampled formulas is correct

In [3]:
print(ft.VerifyFormulaSyntax(formula))

(True, 'AndOr')


Now lets check that the vocabulary of the formula is a subset of the signal vocabulary

In [11]:
print('Formula Vocabulary', ft.GetVocabulary(formula, set()))
print('Signal predicates', set(SampleSignal.keys()) - set(['length']))
print(prog.VerifyVocabulary(formula, SampleSignal))

Formula Vocabulary {'T4', 'P3', 'P2', 'T2', 'T3', 'W1', 'P0', 'W2', 'T0', 'P1', 'W0', 'W3', 'T1'}
Signal predicates {'T4', 'P3', 'P2', 'T2', 'T3', 'W1', 'P0', 'W2', 'T0', 'P1', 'W0', 'W3', 'T1'}
True


Lets generate a slice of the labeling propositions at the first time step

In [23]:
SignalSlice = ft.GetTraceSlice(SampleSignal,0)
print(SignalSlice)

{'T4': False, 'P3': True, 'P2': True, 'T2': False, 'T3': False, 'W1': False, 'P0': True, 'W2': False, 'T0': False, 'P1': True, 'W0': False, 'W3': False, 'T1': False}



The formulas have conditional predicates that make their effect felt from the first time step and are not relevant in the dynamic part of the task, so lets progress the formula by a single time step before beginning the task

In [26]:
progressed_formula = prog.ProgressSingleTimeStep(formula, SignalSlice)
print(progressed_formula)


['and', ['G', ['and', ['not', ['T0']], ['not', ['T1']], ['not', ['T2']], ['not', ['T3']], ['not', ['T4']]]], ['F', ['W0']], ['F', ['W1']], ['F', ['W2']], ['F', ['W3']], ['U', ['not', ['W1']], ['W0']], ['U', ['not', ['W2']], ['W0']], ['U', ['not', ['W2']], ['W1']]]


Note that the progressed formula has a more limited vocabulary compared to the original formula and this is better for progression.

In [22]:
ft.GetVocabulary(progressed_formula, set())

{'T0', 'T1', 'T2', 'T3', 'T4', 'W0', 'W1', 'W2', 'W3'}

We can now use `FindAllProgressions` to evaluate all possible progressions of the given formula and a description of the graph of formulas given the truth assignments in the vocabulary

In [32]:
progressions, edges = prog.FindAllProgressions(progressed_formula)
states = dict([(v,k) for (k,v) in progressions.items()])

Note that as the LTL formulas were stored as nested lists, we have to convert them to json strings to use a dictionary to hash them. `progressions` is a dictionary where the keys represent all possible progressions of the original formula. The value stored at the key represents a unique integer index that can be used as a one-hot encoding 

In [35]:
key = list(progressions.keys())[0]
print(key)

["and", ["G", ["and", ["not", ["T0"]], ["not", ["T1"]], ["not", ["T2"]], ["not", ["T3"]], ["not", ["T4"]]]], ["F", ["W0"]], ["F", ["W1"]], ["F", ["W2"]], ["F", ["W3"]], ["U", ["not", ["W1"]], ["W0"]], ["U", ["not", ["W2"]], ["W0"]], ["U", ["not", ["W2"]], ["W1"]]]


`states` is the inverse map of the states to the LTL formula representing the state

In [36]:
states[0]==key

True

`edges` is a tuple structure that represents the graph of transitions between the LTL formulas and the truth value assignments that cause that transitions

In [46]:
edge = edges[5]
print(f'The source state/formula is:\n {edge[0]}\n\n')
print(f'With truth assignment: \n{edge[2]}\n\n')
print(f'The resulting formula is: \n {edge[1]}')

The source state/formula is:
 ['and', ['G', ['and', ['not', ['T0']], ['not', ['T1']], ['not', ['T2']], ['not', ['T3']], ['not', ['T4']]]], ['F', ['W0']], ['F', ['W1']], ['F', ['W2']], ['F', ['W3']], ['U', ['not', ['W1']], ['W0']], ['U', ['not', ['W2']], ['W0']], ['U', ['not', ['W2']], ['W1']]]


With truth assignment: 
[{'T0': False, 'T1': False, 'T2': False, 'T3': False, 'T4': False, 'W0': True, 'W1': False, 'W2': False, 'W3': True}]


The resulting formula is: 
 ['and', ['G', ['and', ['not', ['T0']], ['not', ['T1']], ['not', ['T2']], ['not', ['T3']], ['not', ['T4']]]], ['F', ['W1']], ['F', ['W2']], ['U', ['not', ['W2']], ['W1']]]


Note that both 'W1' and 'W2' were completed simultaneously
